# Analyze how the batch size affects the performance across installed Caffe variants and models

## Includes

### Standard

In [ ]:
import os
import sys
import json
import re
# from string import Template

### Scientific

In [ ]:
import IPython as ip
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mp

In [ ]:
print ('IPython version: %s' % ip.__version__)
print ('NumPy version: %s' % np.__version__)
print ('SciPy version: %s' % sp.__version__)
print ('Pandas version: %s' % pd.__version__)
print ('Matplotlib version: %s' % mp.__version__)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
# import scipy.stats as st

### Collective Knowledge

In [ ]:
import ck.kernel as ck
print ('CK version: %s' % ck.__version__)

## Access Caffe experimental data

In [ ]:
def get_experimental_results(tags):
    r=ck.access({'action':'search', 'module_uoa':'experiment', 'tags':tags})
    if r['return']>0:
        print ("Error: %s" % r['error'])
        exit(1)
    # FIXME: For now, assume a single entry per the given tags.
    r=ck.access({'action':'list_points', 'module_uoa':'experiment', 'data_uoa': r['lst'][0]['data_uoa']})
    if r['return']>0:
        print ("Error: %s" % r['error'])
        exit(1)
    
    results = {}
    for point in r['points']:
        with open(os.path.join(r['path'], 'ckp-%s.0001.json' % point)) as point_file:
            point_data_raw = json.load(point_file)
            run_info_list = [
                characteristics['run']
                for characteristics in point_data_raw['characteristics_list']
                if characteristics['run']['run_success'] == 'yes'
            ]
            # Select characteristics of interest. TODO: simplify.
            point_data_dict = {
                'time_fw_ms' : [ run_info.get('time_fw_ms',0)      for run_info in run_info_list ],
                #'per_layer'  : [ run_info.get('per_layer_info',[]) for run_info in run_info_list ]
            }
            batch_size = point_data_raw['choices']['env']['CK_CAFFE_BATCH_SIZE']
            results[batch_size] = point_data_dict
    return results

In [ ]:
# All Caffe installations.
caffe_env = ck.access({'action':'show', 'module_uoa':'env', 'tags':'lib,caffe'})
caffe_lst = caffe_env['lst']

# All Caffe models.
caffemodel_env = ck.access({'action':'show', 'module_uoa':'env', 'tags':'caffemodel'})
caffemodel_lst = caffemodel_env['lst']

In [ ]:
first_iteration = True
# For each Caffe installation.
for (caffe, caffe_index) in zip(caffe_lst, range(len(caffe_lst))):
    caffe_tags = re.match('BVLC Caffe framework \((?P<tags>.*)\)', caffe['info']['data_name'])
    caffe_tags = caffe_tags.group('tags').replace(', ', '-')
    # For each Caffe model.
    for (caffemodel, caffemodel_index) in zip(caffemodel_lst, range(len(caffemodel_lst))):
        caffemodel_tags = re.match('Caffe model \(net and weights\) \((?P<tags>.*)\)', caffemodel['info']['data_name'])
        caffemodel_tags = caffemodel_tags.group('tags').replace(', ', '-')
        tags = ','.join([ 'time_gpu', caffe_tags, caffemodel_tags ])
        df_new = pd.DataFrame(data=get_experimental_results(tags)).apply(lambda x: pd.Series(x['time_fw_ms']))
        df_new.columns = [ [caffe_tags]*len(df_new.columns), [caffemodel_tags]*len(df_new.columns), df_new.columns ]
        df_new.columns.names = [ 'caffe', 'caffemodel', 'batch size' ]; df_new.index.names = [ 'repetition' ]
        if first_iteration:
            first_iteration = False
            df_all = df_new
        else:
            df_all = df_all.join(df_new)

## All data indexed by repetitions

In [ ]:
df_all

## All data indexed by installations and repetitions

In [ ]:
df_caffe = df_all.stack(level='caffe').swaplevel('caffe', 'repetition').sortlevel('caffe')
df_caffe

## All data indexed by models and repetitions

In [ ]:
df_caffemodel = df_all.stack(level='caffemodel').swaplevel('caffemodel', 'repetition').sortlevel('caffemodel')
df_caffemodel

## Mean execution time per batch

In [ ]:
df_mean_per_batch = df_all.describe().ix['mean'].unstack(level='batch size')
df_mean_per_batch

In [ ]:
batch_sizes = range(1,len(df_mean_per_batch.columns)+1)

## Mean execution time per image

In [ ]:
df_mean_per_image = df_mean_per_batch / batch_sizes
df_mean_per_image

## Minimum mean execution time per image

In [ ]:
df_mean_per_image.min(axis=1)

In [ ]:
df_mean_per_image.idxmin(axis=1)

In [ ]:
# Is the same answer as via .min(axis=1).values?
# df_mean_per_image.lookup(df_mean_per_image.index, df_mean_per_image.idxmin(axis=1)) == df_mean_per_image.min(axis=1).values

## Select the batch size with the minimum mean execution time per image

In [ ]:
df_mean_per_image_with_repetitions = df_all.stack(['caffe', 'caffemodel']) / batch_sizes
df_mean_per_image_with_repetitions.loc[0]

In [ ]:
num_repetitions = len(df_mean_per_image_with_repetitions.index.levels[0])
df_caffe_caffemodel = pd.DataFrame(
    index=df_mean_per_image_with_repetitions.index, columns=['time (ms)'],
    data=df_mean_per_image_with_repetitions.lookup(
        df_mean_per_image_with_repetitions.index, df_mean_per_image.idxmin(axis=1).tolist()*num_repetitions
    )
).swaplevel('repetition').swaplevel('caffe', 'caffemodel').sortlevel('caffe')
df_caffe_caffemodel.columns.names = ['execution time']
# df_caffe_caffemodel

In [ ]:
df_caffe_caffemodel_mean = df_caffe_caffemodel.groupby(level=['caffe', 'caffemodel']).mean()
df_caffe_caffemodel_std  = df_caffe_caffemodel.groupby(level=['caffe', 'caffemodel']).std()
# df_caffe_caffemodel_mean['time (ms)'] == df_mean_per_image.min(axis=1)

## Plot by Caffe installations

In [ ]:
df_caffe_caffemodel_mean.unstack('caffemodel') \
    .plot(title='Execution time per image (ms)', yerr=df_caffe_caffemodel_std.unstack('caffemodel'), 
          kind='bar', rot=0, ylim=[0,800], figsize=[12, 6], grid=True, legend=True, colormap=cm.autumn)

## Plot by Caffe models

In [ ]:
df_caffe_caffemodel_mean.unstack('caffe') \
    .plot(title='Execution time per image (ms)', yerr=df_caffe_caffemodel_std.unstack('caffe'), 
          kind='bar', rot=0, ylim=[0,800], figsize=[12, 6], grid=True, legend=True, colormap=cm.autumn)